# Podman ecosystem

We have seen what an awsome piece of software Podman is, but that is not all. Podman counts with some other tools in its ecosystem that makes everything run smoothly.
Lets review all of them.

## Managing multiple containers: podman compose and kubernetes

Running a single container with podman is just fine, we have demonstrated how simple it is througout this workshop, but what happens if I want to run multiple containers simultaneously?

You can use the tool podman compose to do that. Podman compose is a tool that helps you defining the list of containers and all of their configuration in a text file written in yaml format. Then you can choose to execute, stop, remove, update and so on all of the containers defined in the file at once. Podman compose is a very powerful tool but it's limited to a single system.

If you're looking at deploying your containers in a number of systems in order to get high availability then you will most probably end up using kubernetes as it has become the standard platform for running containerized workloads in the enterprise.
Kubernetes, same as podman composer, uses yaml files to define the desired state of your containerized workloads and, once you pass the file to the control plane nodes of kubernetes, it will make sure your containers run following your specifications accross as many nodes as you may have.

Now, transitioning from single system container engines like Podman or Docker to kubernetes may not always be easy. That is why Podman can generate a kubernetes yaml file from the container that you are running righ now with Podman. First you just need to execute a new container:

In [1]:
!podman run -d --rm --name=kubepodman docker.io/redhat/ubi9 sleep 999

c39ee8c5bc8cbe0f8c6d064694ecbe984c8a7fac4e0dd16fbd81c3aec209f785


Now your container is running:

In [2]:
!podman ps

CONTAINER ID  IMAGE                         COMMAND     CREATED             STATUS             PORTS       NAMES
c39ee8c5bc8c  docker.io/redhat/ubi9:latest  sleep 999   About a minute ago  Up About a minute              kubepodman


With this, you can execute the "podman generate kube" command to export a yaml file that then you can use for your kubernetes environment.

In [9]:
!podman generate kube kubepodman > kubepodman.yml
!cat kubepodman.yml

# Save the output of this file and use kubectl create -f to import
# it into Kubernetes.
#
# Created with podman-4.7.0
apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: "2023-10-20T11:44:37Z"
  labels:
    app: kubepodman-pod
  name: kubepodman-pod
spec:
  containers:
  - command:
    - sleep
    - "999"
    image: docker.io/redhat/ubi9:latest
    name: kubepodman



The output of the previous command is the content of the yaml file that you need to use for running your container in a kubernetes workload. At the beginning of the file you can even see a few comments in which you find the command you need to run in order to start executing this container in your kubernetes cluster: "kubectl create -f kubepodman.yml".

As you can see, podman has multiple tools that allow you to transition to managing multiple containers at once and simplify the transition to those tools.

## Advanced container images management: Skopeo

Skopeo is a tool for manipulating, inspecting, signing, and transferring container images and image repositories on linux systems, Windows and MacOS. Like Podman and Buildah, Skopeo is an open source community-driven project that does not require running a container daemon.

With Skopeo, you can inspect images on a remote registry without having to download the entire image with all its layers, making it a lightweight and modular solution for working with container images across different formats, including Open Container Initiative (OCI) and Docker images.

Skopeo, Buildah, podman k8s, podman compose